<a href="https://colab.research.google.com/github/rajy4683/S15Test/blob/master/S15Profiling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%%time
'''
Time Taken to unzip 100K elements in parallel from GDrive to Colab's local drive
'''
csv_file ='/content/drive/My Drive/EVA4/tsai/S15EVA4/FinalDataSet/DepthMapDataSetTrain.csv'
root_dir = '/content/drive/My Drive/EVA4/tsai/S15EVA4/'
batch_start = 0
batch_end = batch_start+100000
dest_dir="/content/train"

mmddataloader.extract_data_files_mt(csv_file, root_dir, batch_start,batch_end, dest_dir=dest_dir)


Extracting image and mask files Total:9
Zip Name:image_mask_gt1k2k.zip Total Files:33518
Zip Name:depth_mask_gt_1k2k.zip Total Files:16759
Zip Name:image_mask_gt6k8k.zip Total Files:33154
Zip Name:image_mask_gt8k12k.zip Total Files:67132
Zip Name:image_mask_gt2k6k.zip Total Files:66198
Zip Name:depth_mask_gt_2k6k.zip Total Files:33099
Zip Name:depth_mask_gt_6k8k.zip Total Files:16577
Zip Name:depth_mask_gt_8k12k.zip Total Files:33566
Extracting bg files
Total file count:300113 
CPU times: user 7.69 s, sys: 962 ms, total: 8.65 s
Wall time: 26min 38s


In [0]:
%%time 
'''
Time taken for DataSet object creation
'''
size_hw = (224,224)
dest_dir="/content/train"
train_batch_size = 128
depth_dataset = mmddataloader.DepthMaskDataSet(csv_file, dest_dir, batch_start,batch_end, read_zip=False,
                                 transform_base=mmddataloader.resize_bg(size_hw[0],size_hw[1],[0.56632738, 0.51567622, 0.45670792], [0.1076622, 0.10650349, 0.12808967] ),
                                 transform_bg=mmddataloader.resize_bg(size_hw[0],size_hw[1],[0.57469445, 0.52241555, 0.45992244], [0.11322354, 0.11195428, 0.13441683]),
                                 transform_mask = mmddataloader.resize_bg_nonNorm(size_hw[0],size_hw[1]),
                                 transform_depth = mmddataloader.resize_bg_nonNorm(size_hw[0],size_hw[1]),
                                 #transform_mask=mmddataloader.resize_bg(size_hw[0],size_hw[1], [0.0579508], [0.001662]),
                                 #transform_depth=mmddataloader.resize_bg(size_hw[0],size_hw[1], [0.3679109], [0.03551773]),
                                 )
from torch.utils.data import DataLoader
kwargs= { 'num_workers':8,'pin_memory': True}
train_loader = DataLoader(depth_dataset, batch_size=train_batch_size, shuffle=False, **kwargs)

CPU times: user 856 ms, sys: 106 ms, total: 962 ms
Wall time: 2.06 s


## Dataloader Profiling

In [0]:
def profile_dataloader(data_loader, sample_count=500000):
    pbar = tqdm(data_loader)
    for batch_idx, dataset in enumerate(pbar):
        data_len = len(dataset['input'])
        if(batch_idx > sample_count):
            break